# Модульная архитектура

In [61]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from collections import defaultdict, Counter
import random
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

## Определение Моделей
### Autoencoder

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, latent_dim=512):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            # Слои энкодера
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 8x8
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 4x4
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 2x2
        )
        self.fc_enc = nn.Linear(512 * 2 * 2, latent_dim)
        self.fc_dec = nn.Linear(latent_dim, 512 * 2 * 2)
        self.decoder = nn.Sequential(
            # Слои декодера
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),  # 4x4
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 8x8
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 16x16
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),  # 32x32
            nn.Sigmoid(),
        )

    def forward(self, x):
        z = self.encoder(x)
        z = z.view(z.size(0), -1)
        z = self.fc_enc(z)

        h = self.fc_dec(z)
        h = h.view(z.size(0), 512, 2, 2)
        x_recon = self.decoder(h)
        return x_recon, z

    def encode(self, x, require_grad=True):
        if not require_grad:
            with torch.no_grad():
                z = self.encoder(x)
                z = z.view(z.size(0), -1)
                z = self.fc_enc(z)
        else:
            z = self.encoder(x)
            z = z.view(z.size(0), -1)
            z = self.fc_enc(z)
        return z

### BinaryClassifier

In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self, embedding_dim=512):
        super(BinaryClassifier, self).__init__()
        self.fc = nn.Sequential(nn.Linear(embedding_dim, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, 1))  # Выходной слой для бинарной классификации

    def forward(self, z):
        out = self.fc(z)
        return out  # Без сигмоида, так как используем BCEWithLogitsLoss

### 1.3. CombinedBinaryClassifier

In [ ]:
# Deprecated
class CombinedBinaryClassifier(nn.Module):

    def __init__(self, binary_classifiers, num_classes):

        super(CombinedBinaryClassifier, self).__init__()

        self.binary_classifiers = nn.ModuleList(binary_classifiers)

        self.num_classes = num_classes


    def forward(self, z):

        logits = []

        for classifier in self.binary_classifiers:

            out = classifier(z)

            logits.append(out)

        logits = torch.cat(logits, dim=1)  # [batch, num_classes]

        return logits  # Без активации, будем применять sigmoid позже

## 2. Определение Класса Trainer
Класс Trainer будет управлять процессами обучения автокодировщика, бинарных классификаторов, объединённой модели и дообучения энкодера.

In [ ]:
class Trainer:
    def __init__(self, num_classes=10, latent_dim=512, batch_size=128, device="cuda" if torch.cuda.is_available() else "cpu", save_dir="./saved_models"):
        self.num_classes = num_classes
        self.latent_dim = latent_dim
        self.batch_size = batch_size
        self.device = device
        self.save_dir = save_dir
        os.makedirs(self.save_dir, exist_ok=True)

        # Инициализация моделей
        self.autoencoder = Autoencoder(latent_dim=self.latent_dim).to(self.device)
        self.binary_classifiers = nn.ModuleList([BinaryClassifier(embedding_dim=self.latent_dim).to(self.device) for _ in range(self.num_classes)])

        # Потери и точности
        self.ae_loss_log = []
        self.classifier_loss_log = [[] for _ in range(self.num_classes)]  # Потери для каждого классификатора
        self.classifier_acc_log = [[] for _ in range(self.num_classes)]  # Точности для каждого классификатора
        self.fine_tune_loss_log = []
        self.fine_tune_acc_log = []

        # Точности до и после финетюнинга
        self.acc_combined_before = 0.0
        self.error_rate_before = 1.0
        self.acc_combined_after = 0.0
        self.error_rate_after = 1.0

        # Хранение предсказаний и истинных меток для матрицы ошибок
        self.pred_all_before = [[] for _ in range(self.num_classes)]
        self.target_all_before = [[] for _ in range(self.num_classes)]
        self.pred_all_after = [[] for _ in range(self.num_classes)]
        self.target_all_after = [[] for _ in range(self.num_classes)]

    def prepare_dataloaders(self, train_subset, test_subset):
        self.train_loader = DataLoader(train_subset, batch_size=self.batch_size, shuffle=True, num_workers=2, pin_memory=True)
        self.test_loader = DataLoader(test_subset, batch_size=self.batch_size, shuffle=False, num_workers=2, pin_memory=True)

    def train_autoencoder(self, epochs=10, lr=0.001):
        print("\nИнициализация и обучение автокодировщика...")
        optimizer_ae = optim.Adam(self.autoencoder.parameters(), lr=lr)
        criterion_ae = nn.MSELoss()

        for ep in range(epochs):
            epoch_start = time.time()
            self.autoencoder.train()
            running_loss = 0.0
            for d, _ in self.train_loader:
                d = d.to(self.device)
                optimizer_ae.zero_grad()
                x_recon, z = self.autoencoder(d)
                loss = criterion_ae(x_recon, d)
                loss.backward()
                optimizer_ae.step()
                running_loss += loss.item() * d.size(0)
            epoch_loss = running_loss / len(self.train_loader.dataset)
            self.ae_loss_log.append(epoch_loss)
            epoch_end = time.time()
            print(f"Эпоха {ep+1}/{epochs}, Потери AE: {epoch_loss:.6f}, Время: {epoch_end - epoch_start:.2f} сек.")

    def train_binary_classifiers(self, epochs=10, lr=0.001):
        print("\nОбучение бинарных классификаторов...")
        criterion_cls = nn.BCEWithLogitsLoss()
        optimizers_cls = [optim.Adam(classifier.parameters(), lr=lr, weight_decay=1e-5) for classifier in self.binary_classifiers]

        for ep in range(epochs):
            epoch_start = time.time()
            for cls in range(self.num_classes):
                self.binary_classifiers[cls].train()
                running_loss = 0.0
                correct = 0
                total = 0
                for data, target in self.train_loader:
                    data, target = data.to(self.device), target.to(self.device)
                    optimizer_cls = optimizers_cls[cls]
                    optimizer_cls.zero_grad()

                    z = self.autoencoder.encode(data, require_grad=False)
                    logits = self.binary_classifiers[cls](z).squeeze()
                    binary_targets = (target == cls).float()

                    loss = criterion_cls(logits, binary_targets)
                    loss.backward()
                    optimizer_cls.step()

                    running_loss += loss.item() * data.size(0)

                    preds = (torch.sigmoid(logits) >= 0.5).float()
                    correct += (preds == binary_targets).sum().item()
                    total += data.size(0)
                epoch_loss = running_loss / total
                epoch_acc = correct / total
                self.classifier_loss_log[cls].append(epoch_loss)
                self.classifier_acc_log[cls].append(epoch_acc)
                print(f"Epoch {ep+1}/{epochs}, Class {cls}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%")
            epoch_end = time.time()
            print(f"Эпоха {ep+1}/{epochs} завершена за {epoch_end - epoch_start:.2f} сек.")

    def evaluate_combined_model(self, loader, before_finetune=True):
        if before_finetune:
            pred_all = self.pred_all_before
            target_all = self.target_all_before
        else:
            pred_all = self.pred_all_after
            target_all = self.target_all_after

        for cls in range(self.num_classes):
            pred_all[cls] = []
            target_all[cls] = []

        self.autoencoder.eval()
        for cls in range(self.num_classes):
            self.binary_classifiers[cls].eval()

        with torch.no_grad():
            for data, target in loader:
                data, target = data.to(self.device), target.to(self.device)
                z = self.autoencoder.encode(data, require_grad=False)
                for cls in range(self.num_classes):
                    logits = self.binary_classifiers[cls](z).squeeze()
                    preds = (torch.sigmoid(logits) >= 0.5).float()
                    pred_all[cls].extend(preds.cpu().numpy())
                    binary_targets = (target == cls).float()
                    target_all[cls].extend(binary_targets.cpu().numpy())

        # Расчёт общей точности и error rate
        total_correct = 0
        total_samples = 0
        for cls in range(self.num_classes):
            cls_correct = sum([1 for p, t in zip(pred_all[cls], target_all[cls]) if p == t])
            total_correct += cls_correct
            total_samples += len(target_all[cls])
        accuracy = total_correct / total_samples
        error_rate = 1 - accuracy

        if before_finetune:
            self.acc_combined_before = accuracy
            self.error_rate_before = error_rate
            print(f"\nCombined model accuracy before fine-tuning: {accuracy*100:.2f}%")
            print(f"Error rate before fine-tuning: {error_rate*100:.2f}%")
        else:
            self.acc_combined_after = accuracy
            self.error_rate_after = error_rate
            print(f"\nCombined model accuracy after fine-tuning: {accuracy*100:.2f}%")
            print(f"Error rate after fine-tuning: {error_rate*100:.2f}%")

        return accuracy, error_rate, pred_all, target_all

    def accuracy_per_class(self, pred_all, target_all, before_finetune=True):
        print("\nAccuracy per class:")
        for cls in range(self.num_classes):
            correct = sum([1 for p, t in zip(pred_all[cls], target_all[cls]) if p == t])
            total = len(target_all[cls])
            acc = (correct / total * 100) if total > 0 else 0.0
            print(f"Class {cls}: {acc:.2f}%")

    def save_models(self, suffix="before_finetune"):
        print("\nСохранение моделей...")
        autoenc_save_path = os.path.join(self.save_dir, f"autoencoder_{suffix}.pth")
        torch.save(self.autoencoder.state_dict(), autoenc_save_path)
        print(f"Autoencoder сохранён по пути: {autoenc_save_path}")

        for cls in range(self.num_classes):
            classifier_save_path = os.path.join(self.save_dir, f"classifier_class_{cls}_{suffix}.pth")
            torch.save(self.binary_classifiers[cls].state_dict(), classifier_save_path)
            print(f"BinaryClassifier для класса {cls} сохранён по пути: {classifier_save_path}")

    def load_models(self, suffix="before_finetune"):
        print("\nЗагрузка моделей...")
        autoenc_save_path = os.path.join(self.save_dir, f"autoencoder_{suffix}.pth")
        if os.path.exists(autoenc_save_path):
            self.autoencoder.load_state_dict(torch.load(autoenc_save_path))
            self.autoencoder.to(self.device)
            print(f"Autoencoder загружен из {autoenc_save_path}")
        else:
            print(f"Файл {autoenc_save_path} не найден.")

        for cls in range(self.num_classes):
            classifier_save_path = os.path.join(self.save_dir, f"classifier_class_{cls}_{suffix}.pth")
            if os.path.exists(classifier_save_path):
                self.binary_classifiers[cls].load_state_dict(torch.load(classifier_save_path))
                self.binary_classifiers[cls].to(self.device)
                print(f"BinaryClassifier для класса {cls} загружен из {classifier_save_path}")
            else:
                print(f"Файл {classifier_save_path} не найден.")

    def fine_tune_encoder(self, fine_tune_loader, epochs=3, lr=0.0005):
        print("\nДообучение энкодера на небольшой выборке с фиксированными классификаторами...")
        criterion_fine = nn.BCEWithLogitsLoss()
        optimizer_enc = optim.Adam([p for p in self.autoencoder.parameters() if p.requires_grad], lr=lr, weight_decay=1e-5)

        for ep in range(epochs):
            ep_start = time.time()
            self.autoencoder.train()
            running_loss = 0.0
            correct = 0
            total = 0
            for data, target in fine_tune_loader:
                data, target = data.to(self.device), target.to(self.device).long()
                optimizer_enc.zero_grad()
                z = self.autoencoder.encode(data, require_grad=True)  # Позволяем градиентам проходить
                loss_total = 0.0
                for cls in range(self.num_classes):
                    logits = self.binary_classifiers[cls](z).squeeze()
                    binary_targets = (target == cls).float()
                    loss = criterion_fine(logits, binary_targets)
                    loss_total += loss
                loss_total.backward()
                optimizer_enc.step()

                running_loss += loss_total.item() * data.size(0)

                # Для общей точности агрегируем предсказания
                logits_all = torch.stack([torch.sigmoid(self.binary_classifiers[cls](z).squeeze()) for cls in range(self.num_classes)], dim=1)
                preds_cls = torch.argmax(logits_all, dim=1)
                correct += (preds_cls == target).sum().item()
                total += target.size(0)
            epoch_loss = running_loss / total
            epoch_acc = correct / total
            self.fine_tune_loss_log.append(epoch_loss)
            self.fine_tune_acc_log.append(epoch_acc)
            ep_end = time.time()
            print(f"Fine-tuning Epoch {ep+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%, Время: {ep_end - ep_start:.2f} сек.")

    def plot_metrics(self, before_finetune=True, after_finetune=False):
        if before_finetune:
            # Графики потерь и точности бинарных классификаторов
            fig = make_subplots(rows=2, cols=5, subplot_titles=[f"Class {i} Loss" for i in range(self.num_classes)] + [f"Class {i} Accuracy" for i in range(self.num_classes)])

            for cls in range(self.num_classes):
                # Определение позиции subplot
                row = 1 if cls < 5 else 2
                col = (cls % 5) + 1

                # Потери
                fig.add_trace(go.Scatter(y=self.classifier_loss_log[cls], mode="lines+markers", name=f"Class {cls} Loss"), row=row, col=col)

                # Точность
                fig.add_trace(go.Scatter(y=self.classifier_acc_log[cls], mode="lines+markers", name=f"Class {cls} Accuracy"), row=row + 1 if cls < 5 else row, col=col)

            fig.update_layout(height=800, width=2000, title_text="Binary Classifiers Loss and Accuracy Before Fine-tuning", showlegend=False)
            fig.show()

        if after_finetune:
            # Графики потерь и точности дообучения энкодера
            fig = make_subplots(rows=1, cols=2, subplot_titles=("Fine-tune Loss", "Fine-tune Accuracy"))

            # Потери дообучения
            fig.add_trace(go.Scatter(y=self.fine_tune_loss_log, mode="lines+markers", name="Fine-tune Loss"), row=1, col=1)

            # Точность дообучения
            fig.add_trace(go.Scatter(y=self.fine_tune_acc_log, mode="lines+markers", name="Fine-tune Accuracy"), row=1, col=2)

            fig.update_layout(height=600, width=1000, title_text="Fine-tuning Metrics", showlegend=True)
            fig.show()

    def plot_confusion_matrix(self, before_finetune=True, after_finetune=True):
        if before_finetune:
            for cls in range(self.num_classes):
                cm = confusion_matrix(self.target_all_before[cls], self.pred_all_before[cls], labels=[0, 1])
                fig = go.Figure(data=go.Heatmap(z=cm, x=["Pred 0", "Pred 1"], y=["True 0", "True 1"], colorscale="Blues", showscale=True))
                fig.update_layout(title=f"Confusion Matrix for Class {cls} Before Fine-tuning", xaxis_title="Predicted Label", yaxis_title="True Label")
                fig.show()

        if after_finetune:
            for cls in range(self.num_classes):
                cm = confusion_matrix(self.target_all_after[cls], self.pred_all_after[cls], labels=[0, 1])
                fig = go.Figure(data=go.Heatmap(z=cm, x=["Pred 0", "Pred 1"], y=["True 0", "True 1"], colorscale="Blues", showscale=True))
                fig.update_layout(title=f"Confusion Matrix for Class {cls} After Fine-tuning", xaxis_title="Predicted Label", yaxis_title="True Label")
                fig.show()

## 3. Подготовка Данных
Создадим утилитные функции для подготовки датасетов и загрузчиков.

In [ ]:
def filter_dataset(dataset, num_classes, min_samples):
    """
    Фильтрует датасет, оставляя только `num_classes` классов и минимум `min_samples` образцов на класс.
    """
    class_counts = Counter()
    class_indices = defaultdict(list)

    # Собираем индексы для каждого класса
    for idx, (_, target) in enumerate(dataset):
        if target < num_classes:
            class_indices[target].append(idx)
            class_counts[target] += 1

    # Проверяем, что каждый класс имеет минимум образцов
    for cls in range(num_classes):
        if class_counts[cls] < min_samples:
            raise ValueError(f"Класс {cls} имеет только {class_counts[cls]} образцов, требуется минимум {min_samples}.")

    # Ограничиваем количество образцов до min_samples для каждого класса
    selected_indices = []
    for cls in range(num_classes):
        selected_indices.extend(class_indices[cls][:min_samples])

    return Subset(dataset, selected_indices)


def create_fine_tuning_subset(dataset, num_classes, samples_per_class=200):
    """
    Создаёт выборку для дообучения энкодера с фиксированным количеством образцов на класс.
    """
    selected_indices = []
    class_counts = Counter()
    class_indices = defaultdict(list)

    for idx, (_, target) in enumerate(dataset):
        if target < num_classes and class_counts[target] < samples_per_class:
            class_indices[target].append(idx)
            class_counts[target] += 1
            selected_indices.append(idx)
        if all(count >= samples_per_class for count in class_counts.values()):
            break

    return Subset(dataset, selected_indices)

## 4. Основной Скрипт
Теперь объединим всё вместе в основном скрипте, который будет использовать класс Trainer для выполнения всех этапов обучения, сохранения и загрузки моделей.

### 4.1. Ячейка 1: Обучение Автокодировщика и Бинарных Классификаторов, Сохранение Моделей

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random

# Настройки
num_classes = 10
latent_dim = 512
batch_size = 128
epochs = 10
min_samples_per_class = 200  # Увеличено до 200
max_test_samples = 10000
save_dir = "./saved_models"

# Подготовка трансформаций с аугментацией данных
transform = transforms.Compose(
    [
        transforms.Resize(32),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)

# Загрузка датасетов
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# Фильтрация датасетов
train_subset = filter_dataset(train_dataset, num_classes, min_samples_per_class)
test_subset = filter_dataset(test_dataset, num_classes, min_samples_per_class)

# Ограничение тестового набора
if len(test_subset) > max_test_samples:
    test_indices = random.sample(range(len(test_subset)), max_test_samples)
    test_subset = Subset(test_subset, test_indices)

# Инициализация тренера
trainer = Trainer(num_classes=num_classes, latent_dim=latent_dim, batch_size=batch_size, device="cuda" if torch.cuda.is_available() else "cpu", save_dir=save_dir)

# Подготовка загрузчиков данных
trainer.prepare_dataloaders(train_subset, test_subset)

# Обучение автокодировщика
trainer.train_autoencoder(epochs=epochs, lr=0.001)

# Обучение бинарных классификаторов
trainer.train_binary_classifiers(epochs=epochs, lr=0.001)

# Оценка модели до финетюнинга
trainer.acc_combined_before, trainer.error_rate_before, trainer.pred_all_before, trainer.target_all_before = trainer.evaluate_combined_model(
    trainer.test_loader, before_finetune=True
)

# Подсчёт точности по каждому классу до финетюнинга
trainer.accuracy_per_class(trainer.pred_all_before, trainer.target_all_before, before_finetune=True)

# Сохранение моделей перед финетюнингом
trainer.save_models(suffix="before_finetune")

# Визуализация метрик до финетюнинга
trainer.plot_metrics(before_finetune=True, after_finetune=False)

# Визуализация матрицы ошибок до финетюнинга
trainer.plot_confusion_matrix(before_finetune=True, after_finetune=False)

Files already downloaded and verified
Files already downloaded and verified

Инициализация и обучение автокодировщика...
Эпоха 1/10, Потери AE: 1.536293, Время: 14.65 сек.
Эпоха 2/10, Потери AE: 1.285464, Время: 13.58 сек.
Эпоха 3/10, Потери AE: 1.209843, Время: 13.84 сек.
Эпоха 4/10, Потери AE: 1.191077, Время: 14.08 сек.
Эпоха 5/10, Потери AE: 1.169346, Время: 13.83 сек.
Эпоха 6/10, Потери AE: 1.150507, Время: 15.15 сек.
Эпоха 7/10, Потери AE: 1.129152, Время: 14.55 сек.
Эпоха 8/10, Потери AE: 1.118539, Время: 15.99 сек.
Эпоха 9/10, Потери AE: 1.114847, Время: 14.26 сек.
Эпоха 10/10, Потери AE: 1.109030, Время: 14.71 сек.

Обучение бинарных классификаторов...
Epoch 1/10, Class 0, Loss: 0.5412, Accuracy: 84.40%
Epoch 1/10, Class 1, Loss: 0.4278, Accuracy: 87.80%
Epoch 1/10, Class 2, Loss: 0.4383, Accuracy: 87.00%
Epoch 1/10, Class 3, Loss: 0.4970, Accuracy: 86.75%
Epoch 1/10, Class 4, Loss: 0.5139, Accuracy: 86.05%
Epoch 1/10, Class 5, Loss: 0.4802, Accuracy: 87.65%
Epoch 1/10, Class 

### 4.2. Ячейка 2: Загрузка Моделей и Дообучение Энкодера

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random

# Настройки
num_classes = 10
latent_dim = 512
batch_size = 512  # Уменьшено до 512 для стабильности
fine_tuning_epochs = 10  # Изменено пользователем
samples_per_class = 200  # Увеличено до 200
save_dir = "./saved_models"

# Подготовка трансформаций без аугментации (можно добавить, если необходимо)
transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

# Загрузка датасета для дообучения
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

# Создание выборки для дообучения
fine_tune_subset = create_fine_tuning_subset(train_dataset, num_classes, samples_per_class)
fine_tune_loader = DataLoader(fine_tune_subset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

# Инициализация тренера
trainer = Trainer(num_classes=num_classes, latent_dim=latent_dim, batch_size=batch_size, device="cuda" if torch.cuda.is_available() else "cpu", save_dir=save_dir)

# Загрузка сохранённых моделей
trainer.load_models(suffix="before_finetune")

# Загрузка и фильтрация тестового датасета
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
test_subset = filter_dataset(test_dataset, num_classes, min_samples=200)  # Используем тот же min_samples_per_class

# Подготовка загрузчиков данных (инициализирует test_loader)
trainer.prepare_dataloaders(fine_tune_subset, test_subset)

# Дообучение энкодера
trainer.fine_tune_encoder(fine_tune_loader, epochs=fine_tuning_epochs, lr=0.0005)

# Оценка объединённой модели после дообучения
trainer.acc_combined_after, trainer.error_rate_after, trainer.pred_all_after, trainer.target_all_after = trainer.evaluate_combined_model(trainer.test_loader, before_finetune=False)

# Подсчёт точности по каждому классу после дообучения
trainer.accuracy_per_class(trainer.pred_all_after, trainer.target_all_after, before_finetune=False)

# Сохранение моделей после дообучения
trainer.save_models(suffix="after_finetune")

# Визуализация метрик после дообучения
trainer.plot_metrics(before_finetune=False, after_finetune=True)

# Визуализация матрицы ошибок после дообучения
trainer.plot_confusion_matrix(before_finetune=False, after_finetune=True)

Files already downloaded and verified

Загрузка моделей...
Autoencoder загружен из ./saved_models\autoencoder_before_finetune.pth
BinaryClassifier для класса 0 загружен из ./saved_models\classifier_class_0_before_finetune.pth
BinaryClassifier для класса 1 загружен из ./saved_models\classifier_class_1_before_finetune.pth
BinaryClassifier для класса 2 загружен из ./saved_models\classifier_class_2_before_finetune.pth
BinaryClassifier для класса 3 загружен из ./saved_models\classifier_class_3_before_finetune.pth
BinaryClassifier для класса 4 загружен из ./saved_models\classifier_class_4_before_finetune.pth
BinaryClassifier для класса 5 загружен из ./saved_models\classifier_class_5_before_finetune.pth
BinaryClassifier для класса 6 загружен из ./saved_models\classifier_class_6_before_finetune.pth
BinaryClassifier для класса 7 загружен из ./saved_models\classifier_class_7_before_finetune.pth
BinaryClassifier для класса 8 загружен из ./saved_models\classifier_class_8_before_finetune.pth
Binary

C:\Users\inimatic\AppData\Local\Temp\ipykernel_30612\420597376.py:175: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

C:\Users\inimatic\AppData\Local\Temp\ipykernel_30612\420

Files already downloaded and verified

Дообучение энкодера на небольшой выборке с фиксированными классификаторами...
Fine-tuning Epoch 1/10, Loss: 2.9845, Accuracy: 28.15%, Время: 14.01 сек.
Fine-tuning Epoch 2/10, Loss: 2.8289, Accuracy: 31.45%, Время: 11.94 сек.
Fine-tuning Epoch 3/10, Loss: 2.7034, Accuracy: 35.95%, Время: 11.96 сек.
Fine-tuning Epoch 4/10, Loss: 2.6159, Accuracy: 36.95%, Время: 12.28 сек.
Fine-tuning Epoch 5/10, Loss: 2.5384, Accuracy: 40.05%, Время: 12.08 сек.
Fine-tuning Epoch 6/10, Loss: 2.4696, Accuracy: 42.75%, Время: 12.33 сек.
Fine-tuning Epoch 7/10, Loss: 2.4127, Accuracy: 43.00%, Время: 11.82 сек.
Fine-tuning Epoch 8/10, Loss: 2.3420, Accuracy: 45.50%, Время: 11.66 сек.
Fine-tuning Epoch 9/10, Loss: 2.2884, Accuracy: 45.45%, Время: 11.71 сек.
Fine-tuning Epoch 10/10, Loss: 2.2017, Accuracy: 48.65%, Время: 11.80 сек.

Combined model accuracy after fine-tuning: 90.58%
Error rate after fine-tuning: 9.42%

Accuracy per class:
Class 0: 90.95%
Class 1: 91.40%
Cl